# Gram-Schmidt process

## Instructions
In this assignment you will write a function to perform the Gram-Schmidt procedure, which takes a list of vectors and forms an orthonormal basis from this set.
As a corollary, the procedure allows us to determine the dimension of the space spanned by the basis vectors, which is equal to or less than the space which the vectors sit.

### Matrices in Python
Remember the structure for matrices in *numpy* is,
```python
A[0, 0]  A[0, 1]  A[0, 2]  A[0, 3]
A[1, 0]  A[1, 1]  A[1, 2]  A[1, 3]
A[2, 0]  A[2, 1]  A[2, 2]  A[2, 3]
A[3, 0]  A[3, 1]  A[3, 2]  A[3, 3]
```
You can access the value of each element individually using,
```python
A[n, m]
```
You can also access a whole row at a time using,
```python
A[n]
```

Building on last assignment, in this exercise you will need to select whole columns at a time.
This can be done with,
```python
A[:, m]
```
which will select the m'th column (starting at zero).

In this exercise, you will need to take the dot product between vectors. This can be done using the @ operator.
To dot product vectors u and v, use the code,
```python
u @ v
```

In [1]:
import numpy as np
from numpy import linalg as LA

epsilon = 10e-5

## Gram-Schmidt formulation

Given a basis $\{ v_1, \ldots, v_m\}$ of a subspace $S \subset R^n$, the Gram-Schmidt process
constructs an orthonormal basis $\{ q_1, \ldots, q_m\}$

More precisely, for $k = 1, \ldots, m$ the Gram-Schmidt process successively computes  an orthonormal 
basis $\{ q_1, \ldots, q_k\}$ from $\{ v_1, \ldots, v_k\}$ such that both bases span the same subspace.
The idea is to use orthogonal projection to remove components along the existing basis vectors, 
leaving an orthogonal set.

$q_1 = \frac{a_1}{\lVert{a_1}\rVert}$

$\hat{q_2} = a_2 - (q_1^T a_2)q_1\qquad \Rightarrow \qquad q_2 = \frac{\hat{q_2}}{\lVert{\hat{q_2}}\rVert}$

$\hat{q_k} = a_k - (q_1^T a_k)q_1 - ... - (q_{k-1}^T a_k)q_{k-1}\qquad \Rightarrow \qquad q_k = \frac{\hat{q_k}}{\lVert{\hat{q_2}}\rVert}$


Please compute $\hat{q_{i}}$ in the functions `q_hat_calculate` below where `i` is the `index` variable:

In [2]:
def q_hat_calculate(index, vector, vectors):
    """
    :param index: the i that we want to calculate q_hat for
    :param vector: the ith vector that we want to calculate q_hat for
    :param vectors: the previous q's that we calculated
    :return: q hat vector
    """
    # YOUR CODE HERE
    q_hat = vector.reshape((1, -1))
    for i in range(index):
        q_hat = q_hat - ((q_hat @ vectors[:, i].reshape((-1, 1))) * vectors[:, i].reshape((1, -1)))
        
    return q_hat

Please fill in the functions `gram_schmidt_algorithm` below:

In [3]:
def gram_schmidt_algorithm(vectors, debug=False):
    
    vectors = np.array(vectors, dtype='float32')
    dependent_vectors_index = []  # the index of vectors that are dependent
    orthonormal_basis = np.zeros(shape=vectors.shape, dtype='float32')
    
    column_count = 0 # count the indepentent vectors of matrix `vectors`
    
    for index, column in enumerate(vectors.T):
        # YOUR CODE HERE
        # Compute Gram-Schmidt algorithm
        q_hat = q_hat_calculate(index, column, orthonormal_basis)
        if (q_hat @ q_hat.T) > (epsilon + 0) or (q_hat @ q_hat.T) < (0 - epsilon):
            q = q_hat / (np.linalg.norm(q_hat))
            column_count += 1
            orthonormal_basis[:, index] = q.reshape((-1))
                
        else:
            dependent_vectors_index.append(index)
                         
        

    if debug is True:
        print(f'the input vectors are:\n{vectors}\n')

    if column_count == vectors.shape[1]:  # all vectors were independent
        if debug is True:
            print(f'the vectors were linearly independent and orthonormal basis:\n{orthonormal_basis}')
        return orthonormal_basis, None, None

    if column_count < vectors.shape[1]:  # some vectors were dependent
        # Delete `orthonormal_basis` columns that are 0 (not used)
        # YOUR CODE HERE
        print("dependent index", dependent_vectors_index)
        # Delete dependent vectors from orthonormal basis
        orthonormal_basis = np.delete(orthonormal_basis, dependent_vectors_index, axis=1)
#         raise NotImplementedError
        
        
        # Find the coefficients of dependent vectors and store it in `dependent_combination` matrix
        # colomn i of `dependent_combination` is the coaficients of the the i'th dependent vector)
        # you can use `dependent_vectors_index` for knowing which vectors are dependent
        
        # YOUR CODE HERE
        dependent_combination = np.zeros(shape = (orthonormal_basis.shape[0], len(dependent_vectors_index)), dtype='float32')
        print("dependent com:", dependent_combination.shape)
        for i, j in enumerate(dependent_vectors_index):
            print(f"orth_basis {orthonormal_basis.shape}, vector {vectors[:,j].shape}")
            dependent_combination[:, i] = np.linalg.solve(orthonormal_basis, vectors[:, j])
        
        if debug is True:
            print(f'the vectors were linearly dependent and the orthonormal basis is:\n{orthonormal_basis}')
            print(f'\nthe index of dependent vectors are: {dependent_vectors_index}')
            print(f'\ncoefficients are:\n{dependent_combination}')
        
        return orthonormal_basis, dependent_vectors_index, dependent_combination

# Test GS

In [4]:
# Test1: independent vectors
gram_schmidt_algorithm([[1, 0, 0],
                       [0,  1,  0],
                       [ 0,  0,  1]], True)

the input vectors are:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

the vectors were linearly independent and orthonormal basis:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float32),
 None,
 None)

In [5]:
# Test2: dependent vectors
gram_schmidt_algorithm([[1, 0, 0, 2],
                       [0, 1, 0, 0],
                       [ 0, 0, 1, 0]], True)

the input vectors are:
[[1. 0. 0. 2.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]

dependent index [3]
dependent com: (3, 1)
orth_basis (3, 3), vector (3,)
the vectors were linearly dependent and the orthonormal basis is:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

the index of dependent vectors are: [3]

coefficients are:
[[2.]
 [0.]
 [0.]]


(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float32),
 [3],
 array([[2.],
        [0.],
        [0.]], dtype=float32))

In [6]:
# Test3: 
gram_schmidt_algorithm([[-1, -0, -0],
       [-1,  0,  1],
       [ 2,  3,  4]], True)

the input vectors are:
[[-1.  0.  0.]
 [-1.  0.  1.]
 [ 2.  3.  4.]]

the vectors were linearly independent and orthonormal basis:
[[-4.0824828e-01  5.7735020e-01 -7.0710653e-01]
 [-4.0824828e-01  5.7735020e-01  7.0710701e-01]
 [ 8.1649655e-01  5.7735044e-01 -5.0576222e-07]]


(array([[-4.0824828e-01,  5.7735020e-01, -7.0710653e-01],
        [-4.0824828e-01,  5.7735020e-01,  7.0710701e-01],
        [ 8.1649655e-01,  5.7735044e-01, -5.0576222e-07]], dtype=float32),
 None,
 None)